In [20]:
class Node:
    def __init__(self, state=None, cost=0, hr = 0, parent=None, action=None):
        self.__state= state
        self.__action = action
        self.__parent = parent
        self.__cost = cost
        self.__hr = hr
    @property
    def hr(self):
        return self.__hr
    @property
    def state(self):
        return self.__state

    @property
    def action(self):
        return self.__action

    @property
    def parent(self):
        return self.__parent

    @property
    def cost(self):
        return self.__cost
    
    @hr.setter
    def hr(self, hr):
        self.__hr=hr

    @state.setter
    def status(self, state):
        self.__state = state

    @action.setter
    def action(self, action):
        self.__action = action

    @parent.setter
    def parent(self, parent):
        self.__parent = parent

    @cost.setter
    def cost(self, cost):
        self.__cost = cost
    def __lt__(self, other):
        return self.__hr < other.__hr


In [21]:
import copy 
class Problem:
    def __init__(self, initial_state, goal_state, path_cost):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.path_cost = path_cost

    def goal_test(self, state):
        if state == self.goal_state:
            return True
        else:
            return False

    
    def actions(self, state):
        actions_list = []
        states_list = []
        size = len(state[0])
        for i in range(size):#2
            for j in range(size):#2
                if state[i][j] == 0:#i=0,j=0
                    if i > 0:
                        actions_list.append('UP')
                        states_list.append(self.result('UP', state, i, j))
                    if i + 1 < size:
                        actions_list.append('DOWN')
                        states_list.append(self.result('DOWN', state, i, j))
                    if j > 0:
                        actions_list.append('LEFT')
                        states_list.append(self.result('LEFT', state, i, j))
                    if j + 1 < size:
                        actions_list.append('RIGHT')
                        states_list.append(self.result('RIGHT', state, i, j))
                    break
        return {'states': states_list, 'actions': actions_list}


        
    def result(self, action, state , i , j):
        child_state = copy.deepcopy(state)
        if action == 'UP':
            child_state[i][j], child_state[i - 1][j] = child_state[i-1][j], child_state[i][j]
        elif action == 'DOWN':
            child_state[i][j], child_state[i + 1][j] = child_state[i+1][j], child_state[i][j]
        elif action == 'RIGHT':
            child_state[i][j], child_state[i][j + 1] = child_state[i][j+1], child_state[i][j]
        else:
            child_state[i][j], child_state[i][j - 1] = child_state[i][j-1], child_state[i][j]
        return child_state


In [22]:
def getInvCount(arr):
    size = len(arr)
    i = 0
    count = 0
    while i < size:
        j = i + 1
        while j < size:
            if arr[i] > arr[j] and not arr[j] == 0:
                count = count + 1
            j += 1
        i += 1
    return count
def findXPosition(puzzle):
    size = len(puzzle)
    i = 0
    while i < size:
        j = 0
        while j < size:
            if puzzle[i][j] == 0:
                return size - i
            j += 1
        i += 1
def isSolvable(puzzle):
    arr = []
    size = len(puzzle)
    for i in range(size):
        for j in range(size):
            arr.append(puzzle[i][j])
    count = getInvCount(arr)
    positionX = findXPosition(puzzle)
    if size % 2 == 1:
        return count % 2 == 0
    else:
        if count % 2 == 0:
            return positionX % 2 == 1
        else:
            return positionX % 2 == 0

In [23]:
import sys   
puzzles = []     
with open('problem.txt', 'r') as file:
    N = int(file.readline())
    for _ in range(N):
        line = ([int(x) for x in file.readline().strip().split()])
        line_index = 0
        size = (len(line))**0.5
        size = int(size)
        puzzle = []
        for _ in range(size):
            row = []
            for _ in range(size):
                row.append(line[line_index])
                line_index += 1
            puzzle.append(row)
        puzzles.append(puzzle)



In [24]:
def output_into_file(fileName, result):
    with open(fileName, 'a') as file:
        stdout_original = sys.stdout
        sys.stdout = file
        size = len(result)
        if size:
            for i in range(size - 1):
                print(result[i], end='->')
            print(result[size - 1])
        sys.stdout = stdout_original

In [25]:
def make_goal(size):
    goal = []
    num = 1
    for _ in range(size):
        row = []
        for _ in range(size):
            row.append(num)
            num += 1
        goal.append(row)
    goal[size-1][size-1] = 0
    return goal

def solution(node):
    path = []
    while node.parent:
        path.append(node.action)
        node = node.parent
    return path

def get_hr(state):
    arr = []
    size = len(state)
    for i in range(size):
        for j in range(size):
            arr.append(puzzle[i][j])
    hr_count = getInvCount(arr)
    return hr_count


In [26]:
import heapq as hq
def informed_Search(problem):
    hr = get_hr(problem.initial_state)
    current = Node(problem.initial_state,0, hr + 0 , None, None)
    frontier = []
    hq.heappush(frontier, current)
    exposed = set()
    # if both are same at the beginning no need to check
    if problem.goal_test(current.state):
        solution(current)
    while frontier:
        current = hq.heappop(frontier)
        exposed.add(str(current.state))
        # if both are same at the beginning no need to check
        if problem.goal_test(current.state):
            return solution(current)
        else:
            childs = problem.actions(current.state)
            states = childs['states']
            actions = childs['actions']
            for i in range (len(states)):
                hr = get_hr(states[i])
                fn = current.cost + hr
                node = Node(states[i], current.cost+1, fn , current ,actions[i])
                if not str(node.state) in exposed and not node in frontier:
                   hq.heappush(frontier, node)



In [27]:
for puzzle in puzzles:
    print(puzzle)
    if isSolvable(puzzle):
        goal = make_goal(len(puzzle))
        problem = Problem(puzzle, goal, 0)
        path = list(informed_Search(problem))
        path.reverse()
        output_into_file('output.txt',path )
        for i,action in enumerate(path):
            if not i == len(path)-1:
                print(action, end = "->")
            else:
                print(action)

    else:
        print (" is not solvable")
    input("press any key to continue .......")

[[0, 3], [1, 2]]
 is not solvable
[[0, 3, 2], [1, 6, 4], [8, 7, 5]]


KeyboardInterrupt: 